In [5]:
import os, sys
sys.path.append('..')
import common, sql
import pandas as pd

In [2]:
q='''select * from tool
where id not in (5);'''
tools=sql.execute(q)
tools

[{'id': 1, 'name': 'Maven Security Versions'},
 {'id': 2, 'name': 'Snyk'},
 {'id': 3, 'name': 'Github Dependabot'},
 {'id': 4, 'name': 'OWASP Dependency-Check'},
 {'id': 6, 'name': 'Contrast'},
 {'id': 7, 'name': 'WhiteSource'},
 {'id': 8, 'name': 'Seeker'},
 {'id': 9, 'name': 'Steady'},
 {'id': 10, 'name': 'SourceClear'}]

In [3]:
for tool in tools:
    toolId = tool['id']
    toolName = tool['name']
    q='''select r.id, ifnull(sub.alerts,0) as alerts from repository r
        left join
        (select repositoryId, sum(mA.count)as alerts from mavenAlert mA
        join dependency d on mA.dependencyId = d.id
        where toolId=%s
        group by repositoryId) sub
        on r.id=sub.repositoryId
        where r.id != 44
        order by r.id;'''
    df=pd.DataFrame(sql.execute(q,(toolId,)))
    print(toolName, df.alerts.sum(), df.alerts.median())

Maven Security Versions 3197.0 58.0
Snyk 5073.0 64.0
Github Dependabot 136.0 0.0
OWASP Dependency-Check 12466.0 254.0
Contrast 0.0 0.0
WhiteSource 434.0 0.0
Seeker 0.0 0.0
Steady 2489.0 51.0
SourceClear 2998.0000022649765 70.0


In [4]:
q='''select t.id, t.name, count(distinct p.id)
from mavenAlert mA
join dependency d on mA.dependencyId = d.id
join package p on d.packageId = p.id
join tool t on mA.toolId = t.id
group by t.id,t.name
order by t.id;'''
df=sql.pd_read_sql(q)
df

,id,name,count(distinct p.id)
0,1,Maven Security Versions,36
1,2,Snyk,96
2,3,Github Dependabot,20
3,4,OWASP Dependency-Check,332
4,7,WhiteSource,76
5,9,Steady,91
6,10,SourceClear,107


In [5]:
for tool in tools:
    toolId = tool['id']
    toolName = tool['name']
    q='''select r.id, ifnull(sub.deps,0) as deps from repository r
        left join
        (select repositoryId, count(distinct packageId) as deps from mavenAlert mA
        join dependency d on mA.dependencyId = d.id
        where toolId=%s
        group by repositoryId) sub
        on r.id=sub.repositoryId
        where r.id != 44
        order by r.id;'''
    df=pd.DataFrame(sql.execute(q,(toolId,)))
    print(toolName, df.deps.median())

Maven Security Versions 12.0
Snyk 6.0
Github Dependabot 0.0
OWASP Dependency-Check 38.0
Contrast 0.0
WhiteSource 0.0
Seeker 0.0
Steady 20.0
SourceClear 24.0


In [6]:
q='''select t.id, t.name, count(distinct p.artifact)
from mavenAlert mA
join dependency d on mA.dependencyId = d.id
join package p on d.packageId = p.id
join tool t on mA.toolId = t.id
group by t.id,t.name
order by t.id;'''
df=sql.pd_read_sql(q)
df

,id,name,count(distinct p.artifact)
0,1,Maven Security Versions,14
1,2,Snyk,46
2,3,Github Dependabot,11
3,4,OWASP Dependency-Check,149
4,7,WhiteSource,44
5,9,Steady,39
6,10,SourceClear,53


In [7]:
for tool in tools:
    toolId = tool['id']
    toolName = tool['name']
    q='''select r.id, ifnull(sub.deps,0) as packages from repository r
        left join
        (select repositoryId, count(distinct p.artifact) as deps from mavenAlert mA
        join dependency d on mA.dependencyId = d.id
        join package p on d.packageId = p.id
        where toolId=%s
        group by repositoryId) sub
        on r.id=sub.repositoryId
        where r.id != 44
        order by r.id;'''
    df=pd.DataFrame(sql.execute(q,(toolId,)))
    print(toolName, df.packages.median())

Maven Security Versions 12.0
Snyk 6.0
Github Dependabot 0.0
OWASP Dependency-Check 36.0
Contrast 0.0
WhiteSource 0.0
Seeker 0.0
Steady 19.0
SourceClear 24.0


In [8]:
q='''select t.id, t.name, count(distinct v.id)
from mavenAlert mA
join dependency d on mA.dependencyId = d.id
join package p on d.packageId = p.id
join tool t on mA.toolId = t.id
join vulnerability v on mA.vulnerabilityId = v.id
group by t.id,t.name
order by t.id;'''
df=sql.pd_read_sql(q)
df

,id,name,count(distinct v.id)
0,1,Maven Security Versions,36
1,2,Snyk,189
2,3,Github Dependabot,61
3,4,OWASP Dependency-Check,313
4,7,WhiteSource,146
5,9,Steady,97
6,10,SourceClear,208


In [9]:
for tool in tools:
    toolId = tool['id']
    toolName = tool['name']
    q='''select r.id, ifnull(sub.vuln,0) as vuln from repository r
        left join
        (select repositoryId, count(distinct v.id) as vuln from mavenAlert mA
        join dependency d on mA.dependencyId = d.id
        join package p on d.packageId = p.id
        join vulnerability v on mA.vulnerabilityId = v.id
        where toolId=%s
        group by repositoryId) sub
        on r.id=sub.repositoryId
        where r.id != 44
        order by r.id;'''
    df=pd.DataFrame(sql.execute(q,(toolId,)))
    print(toolName, df.vuln.median())

Maven Security Versions 22.0
Snyk 23.0
Github Dependabot 0.0
OWASP Dependency-Check 117.0
Contrast 0.0
WhiteSource 0.0
Seeker 0.0
Steady 41.0
SourceClear 70.0


In [10]:
q='''select t.id, t.name, count(distinct v.id)
from mavenAlert mA
join dependency d on mA.dependencyId = d.id
join package p on d.packageId = p.id
join tool t on mA.toolId = t.id
join vulnerability v on mA.vulnerabilityId = v.id
where v.cveId is not null and v.sourceId is null
group by t.id,t.name
order by t.id;'''
df=sql.pd_read_sql(q)
df

,id,name,count(distinct v.id)
0,1,Maven Security Versions,36
1,2,Snyk,178
2,3,Github Dependabot,61
3,4,OWASP Dependency-Check,289
4,7,WhiteSource,127
5,9,Steady,89
6,10,SourceClear,187


In [11]:
q='select * from scanMinutes'
df=sql.pd_read_sql(q)
df

,toolId,maven,npm
0,1,3.43333,NaN
1,2,15.13330,1.050000
2,4,14.41670,4.683330
3,5,NaN,0.133333
4,9,385.00000,NaN


In [12]:
q='''select t.name as tool, repositoryId, count(distinct packageId) as c
from mavenAlert mA
join dependency d on mA.dependencyId = d.id
join derivedMavenDependencyTree dMDT on d.id = dMDT.dependencyId
join tool t on mA.toolId = t.id
group by toolId, repositoryId;'''
results = sql.pd_read_sql(q)
for tool, df in results.groupby('tool'):
    print(tool,df.c.median()) 

Github Dependabot 2.0
Maven Security Versions 12.0
OWASP Dependency-Check 23.0
Snyk 6.0
SourceClear 26.0
Steady 20.5
WhiteSource 3.0


In [13]:
q='''select t.name as tool, repositoryId, count(distinct v.id) as c
from mavenAlert mA
join dependency d on mA.dependencyId = d.id
join derivedMavenDependencyTree dMDT on d.id = dMDT.dependencyId
join vulnerability v on mA.vulnerabilityId = v.id
join tool t on mA.toolId = t.id
group by toolId, repositoryId;'''
results = sql.pd_read_sql(q)
for tool, df in results.groupby('tool'):
    print(tool,df.c.median()) 

Github Dependabot 3.0
Maven Security Versions 22.0
OWASP Dependency-Check 93.0
Snyk 27.5
SourceClear 71.0
Steady 41.0
WhiteSource 27.0


In [14]:
q='''select t.name as tool, scope, count(distinct d.packageId) as deps, count(distinct v.id) as vuln
from mavenAlert mA
join dependency d on mA.dependencyId=d.id
join vulnerability v on mA.vulnerabilityId=v.id
join derivedMavenDependencyTree dMDT on d.id = dMDT.dependencyId
join tool t on t.id=mA.toolId
group by toolId, scope;'''
sql.pd_read_sql(q)

,tool,scope,deps,vuln
0,Maven Security Versions,compile,19,27
1,Maven Security Versions,provided,30,35
2,Maven Security Versions,runtime,1,1
3,Maven Security Versions,test,3,2
4,Snyk,compile,56,118
5,Snyk,provided,62,173
6,Snyk,runtime,2,6
7,Snyk,test,25,55
8,Github Dependabot,compile,15,61
9,Github Dependabot,provided,5,5


In [15]:
q='''select t.name as tool, scope, count(distinct d.packageId) as deps, count(distinct v.id) as vuln
from npmAlert nA
join dependency d on nA.dependencyId=d.id
join vulnerability v on nA.vulnerabilityId=v.id
join npmDependencyTree nDT on d.packageId = nDT.packageId
join tool t on t.id=nA.toolId
group by toolId, scope;'''
df=sql.pd_read_sql(q)
df

,tool,scope,deps,vuln
0,Snyk,dev,83,106
1,Snyk,prod,13,25
2,Github Dependabot,dev,8,9
3,Github Dependabot,prod,6,10
4,OWASP Dependency-Check,dev,207,200
5,OWASP Dependency-Check,prod,65,65
6,NPM Audit,dev,51,57
7,NPM Audit,prod,15,12
8,WhiteSource,dev,76,78
9,WhiteSource,prod,12,19


In [16]:
q='''select * from contrast'''
df=sql.pd_read_sql(q)
df

,packageId,language,grade,cveCount,totalClasses,usedClasses
0,102,Java,F,2,435,112
1,105,Java,F,2,348,140
2,61,Java,F,1,216,85
3,85,Java,F,1,502,329
4,2,Java,F,1,721,444
5,83,Java,F,2,265,122
6,104,Java,F,1,40,11
7,52,Java,D,1,3463,1966
8,108,Java,F,1,254,30
9,88,Java,F,32,624,267


In [17]:
df['ratio']=df.usedClasses/df.totalClasses
df

,packageId,language,grade,cveCount,totalClasses,usedClasses,ratio
0,102,Java,F,2,435,112,0.257471
1,105,Java,F,2,348,140,0.402299
2,61,Java,F,1,216,85,0.393519
3,85,Java,F,1,502,329,0.655378
4,2,Java,F,1,721,444,0.615811
5,83,Java,F,2,265,122,0.460377
6,104,Java,F,1,40,11,0.275000
7,52,Java,D,1,3463,1966,0.567716
8,108,Java,F,1,254,30,0.118110
9,88,Java,F,32,624,267,0.427885


In [18]:
df.ratio.median()

0.39351851851851855

In [6]:
q='''select * from srcclrCallChains;'''
df=sql.pd_read_sql(q)
df.callChains.median(), df.callChains.sum()

DatabaseError: Execution failed on sql: select * from srcclrCallChains;
(0, '')
unable to rollback